In [1]:
import pandas as pd
import urllib
from sqlalchemy import create_engine, MetaData, select, exc
from sqlalchemy.orm import sessionmaker

# Custom upload with connection string
from engine_info import server_info
# From normalized_tables.py
from normalized_tables import Product, Container, Inventory, Sales, ProductCombination, MasterDate, db

import warnings
warnings.filterwarnings('ignore')

C:\Users\Selebogo\anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:834: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [2]:
# Creating a connection to MS SQL SERVER
params = urllib.parse.quote_plus(server_info)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)
connection = engine.connect()

In [3]:
# Check what is in the database
engine.table_names()

['Capetown_Fresh_produce_market',
 'container',
 'Durban_Fresh_produce_market',
 'inventory',
 'Joburg_Fresh_produce_combined_cleaned',
 'Joburg_Fresh_produce_commodity_cleaned',
 'Joburg_Fresh_produce_commodity_raw',
 'Joburg_Fresh_produce_container_cleaned',
 'Joburg_Fresh_produce_container_raw',
 'Joburg_Fresh_produce_product_combination_raw',
 'Joburg_Fresh_produce_scrapping_date',
 'master_date',
 'PickNPay_Prices',
 'product',
 'product_combination',
 'sales',
 'Shoprite_Prices',
 'sysdiagrams',
 'woolworths_Prices']

In [4]:
metadata = MetaData(bind=engine)

## 1. Commodity (JHB)

In [5]:
commodity_df = pd.read_sql_table('Joburg_Fresh_produce_commodity_cleaned', con=engine)

In [6]:
commodity_df.head()

,rowid,date,commodity,qty_available,MTD_total_value_sold_(R),total_value_sold_(R),Total_quatity_sold,MTD_Total_quatity_sold,Total_kg_sold,MTD_total_kg_sold
0,1,20 August 2020,AMADUMBE,2,39870.0,0.0,0.0,97.0,0.0,1940.0
1,2,20 August 2020,APPLES,91755,22664221.0,1205932.0,13799.0,261296.0,157462.0,3163863.0
2,3,20 August 2020,ARTICHOKES,1,53100.0,600.0,4.0,439.0,3.0,522.0
3,4,20 August 2020,ASPARAGUS,8,258975.0,34000.0,50.0,359.0,250.0,1795.0
4,5,20 August 2020,ATCHARA,207,1351.2,0.0,0.0,23.0,0.0,65.0


In [7]:
# Check the data type so that it's suited to be inserted in a normalized database
commodity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4433 entries, 0 to 4432
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   rowid                     4433 non-null   int64  
 1   date                      4433 non-null   object 
 2   commodity                 4433 non-null   object 
 3   qty_available             4433 non-null   int64  
 4   MTD_total_value_sold_(R)  4433 non-null   float64
 5   total_value_sold_(R)      4433 non-null   float64
 6   Total_quatity_sold        4433 non-null   float64
 7   MTD_Total_quatity_sold    4433 non-null   float64
 8   Total_kg_sold             4433 non-null   float64
 9   MTD_total_kg_sold         4433 non-null   float64
dtypes: float64(6), int64(2), object(2)
memory usage: 346.5+ KB


In [8]:
# Change the quantity sold to int
commodity_df[['Total_quatity_sold', 'MTD_Total_quatity_sold']] = commodity_df[['Total_quatity_sold', 'MTD_Total_quatity_sold']].astype('int64')

In [9]:
# Convert date column from string to datetime
commodity_df['date'] = pd.to_datetime(commodity_df['date'])

In [10]:
commodity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4433 entries, 0 to 4432
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   rowid                     4433 non-null   int64         
 1   date                      4433 non-null   datetime64[ns]
 2   commodity                 4433 non-null   object        
 3   qty_available             4433 non-null   int64         
 4   MTD_total_value_sold_(R)  4433 non-null   float64       
 5   total_value_sold_(R)      4433 non-null   float64       
 6   Total_quatity_sold        4433 non-null   int64         
 7   MTD_Total_quatity_sold    4433 non-null   int64         
 8   Total_kg_sold             4433 non-null   float64       
 9   MTD_total_kg_sold         4433 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(4), object(1)
memory usage: 346.5+ KB


## 2. Container (JHB)

In [11]:
container_df = pd.read_sql_table('Joburg_Fresh_produce_container_cleaned', con=engine)

In [12]:
container_df.head()

,rowid,date,commodity,container,qty_available,average_price_per_kg,MTD_total_value_sold_(R),total_value_sold_(R),Total_quatity_sold,MTD_Total_quatity_sold,Total_kg_sold,MTD_total_kg_sold
0,1,20 August 2020,AMADUMBE,20KG POCKET,2,0.00,39870.0,0.0,0.0,97.0,0.0,1940.0
1,2,20 August 2020,APPLES,10 X 1KG ECONO PACK CARTON,17,10.00,41932.0,100.0,1.0,547.0,10.0,5470.0
2,3,20 August 2020,APPLES,11KG JUMBLE CARTON,343,6.36,218914.0,1190.0,17.0,3170.0,187.0,34870.0
3,4,20 August 2020,APPLES,12 X 1KG ECONO PACK CARTON,3233,7.86,1454572.0,40738.0,432.0,17353.0,5184.0,208236.0
4,5,20 August 2020,APPLES,12.5KG M6 CARTON,12,0.00,16903.0,0.0,0.0,286.0,0.0,3575.0


In [13]:
container_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15173 entries, 0 to 15172
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   rowid                     15173 non-null  int64  
 1   date                      15173 non-null  object 
 2   commodity                 15173 non-null  object 
 3   container                 15173 non-null  object 
 4   qty_available             15173 non-null  int64  
 5   average_price_per_kg      15173 non-null  float64
 6   MTD_total_value_sold_(R)  15173 non-null  float64
 7   total_value_sold_(R)      15173 non-null  float64
 8   Total_quatity_sold        15173 non-null  float64
 9   MTD_Total_quatity_sold    15173 non-null  float64
 10  Total_kg_sold             15173 non-null  float64
 11  MTD_total_kg_sold         15173 non-null  float64
dtypes: float64(7), int64(2), object(3)
memory usage: 1.4+ MB


In [14]:
# Change the quantity sold to int
container_df[['Total_quatity_sold', 'MTD_Total_quatity_sold']] = container_df[['Total_quatity_sold', 'MTD_Total_quatity_sold']].astype('int64')

In [15]:
# Convert date column from string to datetime
container_df['date'] = pd.to_datetime(container_df['date'])

In [16]:
container_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15173 entries, 0 to 15172
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   rowid                     15173 non-null  int64         
 1   date                      15173 non-null  datetime64[ns]
 2   commodity                 15173 non-null  object        
 3   container                 15173 non-null  object        
 4   qty_available             15173 non-null  int64         
 5   average_price_per_kg      15173 non-null  float64       
 6   MTD_total_value_sold_(R)  15173 non-null  float64       
 7   total_value_sold_(R)      15173 non-null  float64       
 8   Total_quatity_sold        15173 non-null  int64         
 9   MTD_Total_quatity_sold    15173 non-null  int64         
 10  Total_kg_sold             15173 non-null  float64       
 11  MTD_total_kg_sold         15173 non-null  float64       
dtypes: datetime64[ns](

## 3. Combination (JHB)

In [17]:
combo_df = pd.read_sql_table('Joburg_Fresh_produce_combined_cleaned', con=engine, index_col='rowid')

In [18]:
combo_df.head()

,date,commodity,container,unit_mass,product_combination,total_value_sold,total_qty_sold,total_kg_sold,average,highest_price,ave_per_kg,highest_price_per_kg
rowid,,,,,,,,,,,,
1,2020-08-20,AMADUMBE,20KG POCKET,20,"*,*,L,*,*",0.0,0,0.0,0.0,0.0,0.0,0.0
2,2020-08-20,APPLES,10 X 1KG ECONO PACK CARTON,10,"GOLDEN DELICIOUS,CL 1,*,*,*",100.0,1,10.0,100.0,100.0,10.0,10.0
3,2020-08-20,APPLES,10 X 1KG ECONO PACK CARTON,10,"ROYAL GALA,CL 1,*,*,*",0.0,0,0.0,0.0,0.0,0.0,0.0
4,2020-08-20,APPLES,11KG JUMBLE CARTON,11,"GOLDEN DELICIOUS,CL 2,L,*,*",0.0,0,0.0,0.0,0.0,0.0,0.0
5,2020-08-20,APPLES,11KG JUMBLE CARTON,11,"TOPRED,CL 2,L,*,*",0.0,0,0.0,0.0,0.0,0.0,0.0


In [19]:
combo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55423 entries, 1 to 27305
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  55423 non-null  datetime64[ns]
 1   commodity             55423 non-null  object        
 2   container             55423 non-null  object        
 3   unit_mass             55423 non-null  int64         
 4   product_combination   55423 non-null  object        
 5   total_value_sold      55423 non-null  float64       
 6   total_qty_sold        55423 non-null  int64         
 7   total_kg_sold         55423 non-null  float64       
 8   average               55423 non-null  float64       
 9   highest_price         55423 non-null  float64       
 10  ave_per_kg            55423 non-null  float64       
 11  highest_price_per_kg  55423 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(2), object(3)
memory usage: 5.5+ M

In [20]:
combo_df.groupby('container')['commodity'].nunique().sort_values(ascending=False)

container
200G PUNNET PACK BOX    45
4KG BOX                 34
3KG POCKET              33
5KG BOX                 32
500G PUNNET PACK BOX    30
                        ..
4.5KG POCKET             1
4.50KG BOX               1
3KG ECONO PACK           1
3KG BOX 6"MASONITE       1
.125G BOTTLE             1
Name: commodity, Length: 255, dtype: int64

In [21]:
combo_df[combo_df['container'] == '3KG POCKET']['commodity'].unique()[:10]

array(['APPLES', 'AVOCADOS', 'BABY BUTTERNUT', 'BABY GEM SQUASH', 'BEANS',
       'BRINJALS', 'CARROTS', 'CHILLIES', 'GARLIC', 'GEELORA'],
      dtype=object)

There's a many to many relationship between commodity and containers, whereby one commodity can have multiple containers and one container can be of different products.

In [22]:
combo_df.groupby('product_combination')['container'].nunique().sort_values(ascending=False)

product_combination
*,*,*,*,*                        110
*,CL 1,L,*,*                      28
*,*,*,10,*                        24
*,CL 1,M,*,*                      24
*,CL 1,S,*,*                      20
                                ... 
PACKHAM'S TRIUMPH,CL 1,*,38,*      1
PACKHAM'S TRIUMPH,CL 1,*,10,*      1
PACKHAM'S TRIUMPH,*,*,*,*          1
OYSTER,*,*,8,*                     1
*,*, 28,5,*                        1
Name: container, Length: 2730, dtype: int64

In [23]:
combo_df[combo_df['product_combination'] == '*,*,*,*,*']['container'].unique()[:10]

array(['2KG TUB', '4KG TUB', '3KG POCKET', '8KG BOX', '5KG BOX',
       '100G PACKET', '4KG BOX', '2KG POCKET', 'BANANA BOX',
       '5KG BANANA BOX'], dtype=object)

In [24]:
combo_df[combo_df['product_combination'] == '*,*,*,*,*']['commodity'].unique()[:10]

array(['ATCHARA', 'BABY BUTTERNUT', 'BABY ENGLISH CUCUMBER',
       'BABY GEM SQUASH', 'BASIL', 'BEAN SPROUTS', 'BEETROOT',
       'BOK CHOY PAK-CHOI', 'BRINJALS', 'BROCCOLI'], dtype=object)

Similarly, product combination can have multiple containers as well as multiple commodities.

## 4. Add data to normalized tables

### 4.1 product

In [25]:
# Different products in the database
products = combo_df['commodity'].unique()

In [26]:
# Add unique values of products to the product sql table
for item in products:
    
    try:
        
        # Add commodity to database
        commodity = Product(name=item)
        db.session.add(commodity)
        db.session.commit()
    
    except exc.IntegrityError:
        
        db.session.rollback()

### 4.2 container

In [27]:
# Different containers in the database
containers = container_df['container'].unique()

In [28]:
# Add unique values of containers to the container sql table
for package in containers:
    
    try:
        
        # Add commodity to database
        container = Container(name=package)
        db.session.add(container)
        db.session.commit()
    
    except exc.IntegrityError:
        
        db.session.rollback()

### 4.3 product_combination

In [29]:
# Different product combinations in the database
combinations = combo_df['product_combination'].unique()

In [30]:
# Add unique values of product combinations to the product combinations sql table
for combo in combinations:
    
    try:
        
        # Add commodity to database
        product_combo = ProductCombination(name=combo)
        db.session.add(product_combo)
        db.session.commit()
    
    except exc.IntegrityError:
        
        db.session.rollback()

### 4.4 master_date

In [31]:
# Dates that are in the database
dates = combo_df['date'].unique()

In [32]:
mydates = pd.to_datetime(dates)

In [33]:
for mydate in mydates:
    
    try:
        
        date_info = MasterDate(
            date_key=mydate,
            month=mydate.month_name(),
            week_of_year=mydate.weekofyear,
            day=mydate.day_name()
        )
        db.session.add(date_info)
        db.session.commit()
    
    except exc.IntegrityError:
        
        db.session.rollback()

### 4.5 inventory

In [34]:
# The inventory table will consist of the products not sold for that day
container_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15173 entries, 0 to 15172
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   rowid                     15173 non-null  int64         
 1   date                      15173 non-null  datetime64[ns]
 2   commodity                 15173 non-null  object        
 3   container                 15173 non-null  object        
 4   qty_available             15173 non-null  int64         
 5   average_price_per_kg      15173 non-null  float64       
 6   MTD_total_value_sold_(R)  15173 non-null  float64       
 7   total_value_sold_(R)      15173 non-null  float64       
 8   Total_quatity_sold        15173 non-null  int64         
 9   MTD_Total_quatity_sold    15173 non-null  int64         
 10  Total_kg_sold             15173 non-null  float64       
 11  MTD_total_kg_sold         15173 non-null  float64       
dtypes: datetime64[ns](

In [35]:
for index, row in container_df.iterrows():
    
    some_product = Product.query.filter_by(name=row['commodity']).first()
    some_container = Container.query.filter_by(name=row['container']).first()
    some_date = MasterDate.query.filter_by(date_key=row['date']).first()
    
    try:
        
        inventory = Inventory(
            inventory_date=some_date,
            available=row['qty_available'],
            product_inventory=some_product,
            container_inventory=some_container
        )
        db.session.add(inventory)
        db.session.commit()
    
    except exc.IntegrityError:
        
        db.session.rollback()

### 4.6 sales

In [36]:
combo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55423 entries, 1 to 27305
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  55423 non-null  datetime64[ns]
 1   commodity             55423 non-null  object        
 2   container             55423 non-null  object        
 3   unit_mass             55423 non-null  int64         
 4   product_combination   55423 non-null  object        
 5   total_value_sold      55423 non-null  float64       
 6   total_qty_sold        55423 non-null  int64         
 7   total_kg_sold         55423 non-null  float64       
 8   average               55423 non-null  float64       
 9   highest_price         55423 non-null  float64       
 10  ave_per_kg            55423 non-null  float64       
 11  highest_price_per_kg  55423 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(2), object(3)
memory usage: 5.5+ M

In [37]:
# Exclude products that were not sold
filtered_df = combo_df[combo_df['total_qty_sold'] != 0]

In [38]:
for index, row in filtered_df.iterrows():
    
    some_product = Product.query.filter_by(name=row['commodity']).first()
    some_container = Container.query.filter_by(name=row['container']).first()
    some_combo = ProductCombination.query.filter_by(name=row['product_combination']).first()
    some_date = MasterDate.query.filter_by(date_key=row['date']).first()
    
    invoice = Sales(
        sales_date=some_date,
        quantity_sold=row['total_qty_sold'],
        kg_sold=row['total_kg_sold'],
        value=row['total_value_sold'],
        average_price=row['average'],
        highest_price=row['highest_price'],
        combination_sale=some_combo,
        container_sale=some_container,
        product_sale=some_product
    )
    
    db.session.add(invoice)
    db.session.commit()

In [39]:
db.session.close()

In [40]:
connection.close()